## I. STOCK PREVISION Up to 30 DAYS (n_forecast) in the future

In [1]:
# https://stackoverflow.com/questions/69906416/forecast-future-values-with-lstm-in-python

from bs4 import BeautifulSoup
import time
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, TimeDistributed, RepeatVector, Flatten
from keras.callbacks import EarlyStopping

from math import ceil
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display

pd.options.mode.chained_assignment = None
tf.random.set_seed(0)

# import warnings
# warnings.filterwarnings('ignore')

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

# download the data
tickerName = 'OR.PA'
periodTime = '20y'
df = yf.download(tickers=[tickerName], period='1y')
y = df['Close'].fillna(method='ffill')
y = y.values.reshape(-1, 1)
df

# ['AI.PA', 'KER.PA', 'OR', 'RI.PA', 'RIO']



[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-08-01,367.549988,372.149994,364.049988,369.250000,359.018127,314868
2022-08-02,370.000000,370.500000,363.799988,363.799988,353.719116,335689
2022-08-03,361.850006,364.200012,358.799988,361.399994,351.385620,367074
2022-08-04,362.450012,368.000000,361.799988,366.350006,356.198486,234931
2022-08-05,366.850006,368.850006,356.799988,358.149994,348.225677,282226
...,...,...,...,...,...,...
2023-07-25,412.000000,415.299988,407.149994,409.950012,409.950012,286702
2023-07-26,410.299988,411.500000,401.500000,407.049988,407.049988,338555
2023-07-27,409.950012,415.950012,408.500000,412.649994,412.649994,375624


In [2]:
# Compute RSI


In [3]:
# ==================================================================
# 2. Scale the data
# ==================================================================
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(y)
y = scaler.transform(y)



# ==================================================================
# 3. Generate the input and output sequences
# ==================================================================
n_forecast = 7  # length of output sequences (forecast period)
n_lookback = n_forecast*20  # length of input sequences (lookback period)


X = []
Y = []

for i in range(n_lookback, len(y) - n_forecast + 1):
    X.append(y[i - n_lookback: i])
    Y.append(y[i: i + n_forecast])

X = np.array(X)
Y = np.array(Y)



# ==================================================================
# 4. Fit the model
# ==================================================================
number_of_features = n_forecast
window_length = n_lookback

# model = Sequential()
# model.add(LSTM(units=50, return_sequences=True, input_shape=(n_lookback, 1)))
# model.add(LSTM(units=50))
# model.add(Dense(n_forecast))


# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(480,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.25))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(480,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.25))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(480,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(480,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.25))
# Adding the first output layer
model.add(Dense(59))
# Adding the last output layer
model.add(Dense(number_of_features))


In [ ]:
# Moniteur pour arreter le training 
es = EarlyStopping(monitor='loss', mode='max', verbose=1, patience=20, restore_best_weights=True) #patience=100)

# Compile & Fit model
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(X, Y, epochs=100, batch_size=32, verbose=2, callbacks =[es])

Epoch 1/100


In [ ]:
# Plot Loss Function
plt.plot(history.history['loss'])
plt.legend(['train_loss'])
plt.show()

In [ ]:
# ==================================================================
# 5. Generate the forecasts
# ==================================================================
X_ = y[- n_lookback:]  # last available input sequence
X_ = X_.reshape(1, n_lookback, 1)

Y_ = model.predict(X_).reshape(-1, 1)
Y_ = scaler.inverse_transform(Y_)



# ==================================================================
# 6. Organize the results in a data frame
# ==================================================================
df_past = df[['Close']].reset_index()
df_past.rename(columns={'index': 'Date', 'Close': 'Actual'}, inplace=True)
df_past['Date'] = pd.to_datetime(df_past['Date'])
df_past['Forecast'] = np.nan
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
df_future['Forecast'] = Y_.flatten()
df_future['Actual'] = np.nan

results = df_past.append(df_future).set_index('Date')



# ==================================================================
# 7. Plot the results
# ==================================================================
#%matplotlib notebook
results.plot(title=tickerName)


In [ ]:
df_future['Forecast'].plot(title=tickerName)

In [ ]:
# ==================================================================
# 8. Display the table results
# ==================================================================
df_future.drop(['Actual'], axis=1, inplace=True)
print(df_future)

## II. Create Overbought - Oversold AREA

In [ ]:
from datetime import datetime
import matplotlib
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import seaborn as sns
sns.set()

df = df.reset_index()
date = df['Date']

In [ ]:
candlesticks = list(zip(mdates.date2num(date),df['Open'],
                   df['High'],df['Low'],df['Close'],df['Volume']))

In [ ]:
def removal(signal, repeat):
    copy_signal = np.copy(signal)
    for j in range(repeat):
        for i in range(3, len(signal)):
            copy_signal[i - 1] = (copy_signal[i - 2] + copy_signal[i]) / 2
    return copy_signal

def get(original_signal, removed_signal):
    buffer = []
    for i in range(len(removed_signal)):
        buffer.append(original_signal[i] - removed_signal[i])
    return np.array(buffer)

signal = np.copy(df.Open.values)
removed_signal = removal(signal, 30)
noise_open = get(signal, removed_signal)

signal = np.copy(df.High.values)
removed_signal = removal(signal, 30)
noise_high = get(signal, removed_signal)

signal = np.copy(df.Low.values)
removed_signal = removal(signal, 30)
noise_low = get(signal, removed_signal)

signal = np.copy(df.Close.values)
removed_signal = removal(signal, 30)
noise_close = get(signal, removed_signal)

In [ ]:
noise_candlesticks = list(zip(mdates.date2num(date),noise_open,
                              noise_high,noise_low,noise_close))


fig = plt.figure(figsize = (15, 12))
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)

ax1.set_ylabel('Quote ($)', size=20)

dates = [x[0] for x in candlesticks]
dates = np.asarray(dates)
volume = [x[5] for x in candlesticks]
volume = np.asarray(volume)

candlestick_ohlc(ax1, candlesticks, width=1,
                 colorup='g', colordown='r')
pad = 0.25
yl = ax1.get_ylim()
ax1.set_ylim(yl[0]-(yl[1]-yl[0])*pad,yl[1])
ax2 = ax1.twinx()

pos = df['Open'] - df['Close']<0
neg = df['Open'] - df['Close']>0
ax2.bar(dates[pos],volume[pos],color='green',width=1,align='center')
ax2.bar(dates[neg],volume[neg],color='red',width=1,align='center')

ax2.set_xlim(min(dates),max(dates))
yticks = ax2.get_yticks()
ax2.set_yticks(yticks[::3])

ax2.yaxis.set_label_position("right")
ax2.set_ylabel('Volume', size=20)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax1.xaxis.set_major_locator(mticker.MaxNLocator(10))

ax2 = plt.subplot2grid((3, 1), (2, 0))

ax2.set_ylabel('Quote ($)', size=20)

candlestick_ohlc(ax2, noise_candlesticks, width=1,
                 colorup='g', colordown='r')
ax2.plot(dates, [np.percentile(noise_close, 95)] * len(noise_candlesticks), color = (1.0, 0.792156862745098, 0.8, 1.0),
       linewidth=5.0, label = 'overbought line')

ax2.plot(dates, [np.percentile(noise_close, 10)] * len(noise_candlesticks), 
        color = (0.6627450980392157, 1.0, 0.6392156862745098, 1.0),
       linewidth=5.0, label = 'oversold line')

ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax2.xaxis.set_major_locator(mticker.MaxNLocator(10))

plt.legend()
plt.show()